In [20]:
import tensorflow as tf
import numpy as np

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

## Bandit

In [26]:
#밴딧의 손잡이 목록 작성
#현재 손잡이 4가 가장 + 보상이 좋도록 설정되어 있음 
bandits = [0.2, 0, -0.2, -2]
num_bandits = len(bandits)

def pull_bandit(bandit):
    result = np.random.randn(1) #랜덤한 표준 정규분포값 생성
    if result > bandit:
        return 1
    else:
        return -1

# Agent

In [27]:
tf.compat.v1.reset_default_graph()

#네트워크의 피드포워드 부분 구현
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,0)


reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)

#학습 과정을 구현
#보상과 선택된 액션을 네트워크에 피드백함으로, 비용을 계산하고 비용을 이용해 네트워크를 업데이트
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

# Training Agent

In [28]:
#에이전트를 학습시킬 총 에피소드의 수를 구함
total_episodes = 1000

#밴딧 손잡이에 대한 점수판을 0으로 설정
total_reward = np.zeros(num_bandits)

e = 0.1 #랜덤 액션의 확률분포 값

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
            
        #Get our reward from picking one of the bandits.     
        reward = pull_bandit(bandits[action]) 
        
        #Update the network
        _, resp, ww = sess.run([update, responsible_weight, weights],\
                               feed_dict={reward_holder:[reward], action_holder:[action]})
        
        #보상의 총량 업데이트
        total_reward[action] += reward
        if i % 50 == 0:
            print("Running reward for the " + str(num_bandits) +" bandits:" + str(total_reward))
        i += 1
        

print("\n The agent thinks bandit " + str(np.argmax(ww) + 1) + " is the most promising.")
if np.argmax(ww) == np.argmax(-np.array(bandit_arms)):
    print("...and it was right")
else:
    print("...and it was wrong")

Running reward for the 4 bandits:[-1.  0.  0.  0.]
Running reward for the 4 bandits:[-3. -5. -1. 30.]
Running reward for the 4 bandits:[-3. -5. -2. 77.]
Running reward for the 4 bandits:[ -1.  -5.  -2. 117.]
Running reward for the 4 bandits:[ -1.  -4.  -2. 164.]
Running reward for the 4 bandits:[  1.  -7.  -1. 208.]
Running reward for the 4 bandits:[  1.  -9.   0. 255.]
Running reward for the 4 bandits:[  0. -10.   0. 301.]
Running reward for the 4 bandits:[  0. -10.   0. 345.]
Running reward for the 4 bandits:[  0.  -8.   1. 390.]
Running reward for the 4 bandits:[  0.  -8.   1. 436.]
Running reward for the 4 bandits:[ -1.  -6.   1. 477.]
Running reward for the 4 bandits:[ -2.  -6.   1. 518.]
Running reward for the 4 bandits:[ -1.  -7.   2. 565.]
Running reward for the 4 bandits:[  0.  -7.   2. 610.]
Running reward for the 4 bandits:[ -1.  -7.   2. 655.]
Running reward for the 4 bandits:[ -1.  -6.   3. 699.]
Running reward for the 4 bandits:[  0.  -8.   4. 743.]
Running reward for the